In [17]:
import os, shutil
import random
from keras import (models, layers, optimizers)
from keras.preprocessing.image import ImageDataGenerator

In [10]:
data_path = "../data/raw"
x_train = os.listdir(data_path + "/train")
x_test = os.listdir(data_path + "/test1")
x_train = [os.path.join(data_path, "train", x) for x in x_train]
x_test = [os.path.join(data_path, "test1", x) for x in x_test]
print(len(x_train))
print(len(x_test))
print(x_train[:5])
print(x_test[:5])

25000
12500
['../data/raw/train/dog.11082.jpg', '../data/raw/train/cat.2955.jpg', '../data/raw/train/cat.12060.jpg', '../data/raw/train/dog.971.jpg', '../data/raw/train/cat.6966.jpg']
['../data/raw/test1/8162.jpg', '../data/raw/test1/262.jpg', '../data/raw/test1/7756.jpg', '../data/raw/test1/4163.jpg', '../data/raw/test1/4979.jpg']


In [11]:
x_valid = random.choices(x_train, k=250)
x_train = random.choices(x_train, k=2000)
x_test = random.choices(x_train, k=250)
print(len(x_test))

250


In [18]:
data_path = "../data/processed"
if os.path.exists(data_path):
    shutil.rmtree(data_path)
os.makedirs(data_path + "/train/dogs")
os.makedirs(data_path + "/train/cats")
os.makedirs(data_path + "/valid/dogs")
os.makedirs(data_path + "/valid/cats")

In [19]:
for x in x_train:
    if "dog" in x:
        shutil.copy(x, data_path + "/train/dogs")
    else:
        shutil.copy(x, data_path + "/train/cats")
for x in x_valid:
    if "dog" in x:
        shutil.copy(x, data_path + "/valid/dogs")
    else:
        shutil.copy(x, data_path + "/valid/cats")

In [20]:
print(len(os.listdir(data_path + "/train/dogs")))
print(len(os.listdir(data_path + "/train/cats")))
print(len(os.listdir(data_path + "/valid/dogs")))
print(len(os.listdir(data_path + "/valid/cats")))

925
1000
119
130


In [12]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

In [13]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

In [14]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

In [21]:
x_train_gen = train_gen.flow_from_directory(
    data_path + "/train",
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)
x_valid_gen = test_gen.flow_from_directory(
    data_path + "/train",
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

Found 1925 images belonging to 2 classes.
Found 1925 images belonging to 2 classes.
